In [ ]:
import cv2,os

data_path='/content/drive/MyDrive/Dataset Macem-macem/DATASET COVID19 NEW/dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'Covid19 Positive': 0, 'Covid19 Negative': 1}
['Covid19 Positive', 'Covid19 Negative']
[0, 1]


In [ ]:
img_size=128
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.

In [ ]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [ ]:
data.shape

(682, 128, 128, 1)

In [ ]:
np.save('/content/drive/MyDrive/Dataset Macem-macem/DATASET COVID19 NEW/data',data)
np.save('/content/drive/MyDrive/Dataset Macem-macem/DATASET COVID19 NEW/target',new_target)

NameError: ignored

In [ ]:
import numpy as np

data=np.load('/content/drive/MyDrive/Dataset Macem-macem/DATASET COVID19 NEW/data.npy')
target=np.load('/content/drive/MyDrive/Dataset Macem-macem/DATASET COVID19 NEW/target.npy')

In [ ]:
target.shape

(682, 2)

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Activation,MaxPooling2D
from keras.utils import normalize
from keras.layers import Concatenate
from keras import Input
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import keras
import tensorflow

input_shape=data.shape[1:]
inp=Input(shape=input_shape)
convs=[]

parrallel_kernels=[3,5,7]

for k in range(len(parrallel_kernels)):

    conv = Conv2D(128, parrallel_kernels[k],padding='same',activation='relu',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)

out = Concatenate()(convs)
conv_model = Model(inp, out)

model = Sequential()
model.add(conv_model)

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,input_dim=64,activation='softmax'))
adam = tensorflow.keras.optimizers.Adam(lr=0.0001,epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 128, 128, 384)     11008     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 126, 126, 64)      221248    
_________________________________________________________________
activation (Activation)      (None, 126, 126, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.0001)
earlystop = EarlyStopping(monitor='val_loss', patience=5)
history=model.fit(train_data,train_target,epochs=20,callbacks=[reduce_lr,earlystop,checkpoint],validation_split=0.2)

Epoch 1/20
16/16 [==============================] - 2s 98ms/step - loss: 0.2954 - accuracy: 0.9122 - val_loss: 0.3473 - val_accuracy: 0.9024
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/20
16/16 [==============================] - 2s 95ms/step - loss: 0.3144 - accuracy: 0.9020 - val_loss: 0.3719 - val_accuracy: 0.8780
Epoch 3/20
16/16 [==============================] - 2s 95ms/step - loss: 0.2909 - accuracy: 0.9082 - val_loss: 0.3415 - val_accuracy: 0.8943
INFO:tensorflow:Assets written to: model-003.model/assets
Epoch 4/20
16/16 [==============================] - 2s 95ms/step - loss: 0.2758 - accuracy: 0.9082 - val_loss: 0.3745 - val_accuracy: 0.8943
Epoch 5/20
16/16 [==============================] - 2s 96ms/step - loss: 0.2754 - accuracy: 0.9041 - val_loss: 0.3337 - val_accuracy: 0.9024
INFO:tensorflow:Assets written to: model-005.model/assets
Epoch 6/20
16/16 [==============================] - 2s 96ms/step - loss: 0.2814 - accuracy: 0.9000 - val_loss: 0.3791 - v